In [1]:
import pandas as pd
import numpy as np

In [60]:
product_types = """
Проезд в трамвае, поездка
Проезд в троллейбусе, поездка
Проезд в метро, поездка
Проезд в городском автобусе,поездка
Полет в салоне экономического класса самолета, в расчете на 1000 км пути
Водоснабжение горячее, м3
Отопление, Гкал
Водоотведение, м3
Газ сжиженный, м3
Проживание в гостинице 1* или в мотеле, сутки с человека
Проживание в гостинице 2* , сутки с человека
Проживание в гостинице 3* , сутки с человека
Проживание в гостинице 4*-5* , сутки с человека
Проживание в хостеле, сутки с человека
Услуги по снабжению электроэнергией
Санаторий, день
Дом отдыха, пансионат, день
""".split("\n")

categories = """
Общественный транспорт
Общественный транспорт
Общественный транспорт
Общественный транспорт
Общественный транспорт
Коммунальные услуги
Коммунальные услуги
Коммунальные услуги
Коммунальные услуги
Места размещения
Места размещения
Места размещения
Места размещения
Места размещения
Коммунальные услуги
Места размещения
Места размещения
""".split("\n")

product_types = [item for item in product_types if len(item) > 0]
categories = [item for item in categories if len(item) > 0]


In [2]:
product_types = pd.read_excel("./data/test2/types.xlsx")
product_types_dict = {product_types['product_types'].iloc[i]:product_types['category'].iloc[i].strip() for i in range(len(product_types['product_types'])) }

product_types_dict

{'Брюки для детей школьного возраста из джинсовой ткани, шт.': 'Одежда',
 'Пеленки для новорожденных, шт.': 'Детские товары',
 'Костюм спортивный для детей школьного возраста, шт.': 'Одежда',
 'Майка, футболка мужская бельевая, шт.': 'Одежда',
 'Футболка детская, шт.': 'Одежда',
 'Колготки женские эластичные, шт.': 'Одежда',
 'Носки мужские, пара': 'Одежда',
 'Кроссовые туфли для детей, пара': 'Обувь',
 'Кроссовые туфли для взрослых, пара': 'Обувь',
 'Мыло хозяйственное, 200 г': 'Гигиена',
 'Порошок стиральный, кг': 'Гигиена',
 'Мыло туалетное, 100 г': 'Гигиена',
 'Шампунь, 250 мл': 'Гигиена',
 'Паста зубная, 100 г (100 мл)': 'Гигиена',
 'Щетка зубная, шт.': 'Гигиена',
 'Сигареты с фильтром, пачка': 'Алкогольная и табачная продукция',
 'Спички, коробок': 'Товары для дома',
 'Электропылесос напольный, шт.': 'Электроника',
 'Бумага туалетная, рулон': 'Гигиена',
 'Подгузники детские бумажные, 10 шт.': 'Детские товары',
 'Прокладки женские гигиенические, 10 шт.': 'Гигиена',
 'Телевизор, шт

In [61]:
product_types_dict = {product_types[i]: categories[i] for i in range(len(categories))}

product_types_dict

{'Проезд в трамвае, поездка': 'Общественный транспорт',
 'Проезд в троллейбусе, поездка': 'Общественный транспорт',
 'Проезд в метро, поездка': 'Общественный транспорт',
 'Проезд в городском автобусе,поездка': 'Общественный транспорт',
 'Полет в салоне экономического класса самолета, в расчете на 1000 км пути': 'Общественный транспорт',
 'Водоснабжение горячее, м3': 'Коммунальные услуги',
 'Отопление, Гкал': 'Коммунальные услуги',
 'Водоотведение, м3': 'Коммунальные услуги',
 'Газ сжиженный, м3': 'Коммунальные услуги',
 'Проживание в гостинице 1* или в мотеле, сутки с человека': 'Места размещения',
 'Проживание в гостинице 2* , сутки с человека': 'Места размещения',
 'Проживание в гостинице 3* , сутки с человека': 'Места размещения',
 'Проживание в гостинице 4*-5* , сутки с человека': 'Места размещения',
 'Проживание в хостеле, сутки с человека': 'Места размещения',
 'Услуги по снабжению электроэнергией': 'Коммунальные услуги',
 'Санаторий, день': 'Места размещения',
 'Дом отдыха, панс

In [65]:
dataset = pd.read_excel("./data/Исходник_Для_парсера_УслугиСредние_потребительские_цены_на_услуги.xlsx")
type(dataset.iloc[6, 1]) is str

True

In [60]:
dataset = pd.read_excel("./data/test2/Средние_потребительские_цены_на_непродовольственные_товарыСредние.xlsx")
dataset.iloc[6, 1]

'01.07.2022\xa0'

In [64]:
dataset = pd.read_excel("./data/1/Исходник Индекс_потребительских_цен_ИПЦ_непродыИндекс_потребительских_цен (1).xlsx")
# dataset = pd.read_excel("./data/2/Исходник Индекс_потребительских_цен_ИПЦ_Основные_группыИндекс_потребительских.xlsx")
type(dataset.iloc[6, 1]) is float

True

225.23

In [49]:
area_coords = []

for i in range(len(dataset.columns)):
	item = dataset.iloc[2, i]
	item = str(item)
	if len(item) > 5:
		if len(area_coords) > 0:
			area_coords[-1].append([2, i])
		area_coords.append(
			[
				item,
				[2, i]
			]
		)
area_coords[-1].append([2, len(dataset.columns)])

area_coords.insert(0,
	['', [2, 1], [2, area_coords[0][1][-1]]]
)

In [22]:
area_coords

[['', [2, 1], [2, 2]],
 ['Центральный федеральный округ', [2, 2], [2, 21]],
 ['Северо-Западный федеральный округ', [2, 21], [2, 33]],
 ['Южный федеральный округ', [2, 33], [2, 42]],
 ['Северо-Кавказский федеральный округ', [2, 42], [2, 50]],
 ['Приволжский федеральный округ', [2, 50], [2, 65]],
 ['Уральский федеральный округ', [2, 65], [2, 72]],
 ['Сибирский федеральный округ', [2, 72], [2, 83]],
 ['Дальневосточный федеральный округ', [2, 83], [2, 95]]]

In [7]:
product_types_set = list(set(product_types_dict.values()))
product_types_set

['Товары для дома',
 'Алкогольная и табачная продукция',
 'Обувь',
 'Детские товары',
 'Гигиена',
 'Медикаменты',
 'Автотовары',
 'Транспорт',
 'Одежда',
 'Электроника']

In [35]:
len(dataset)
dataset.head(20)
# dataset.iloc[12, 0]

,Индекс потребительских цен,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94
0,Российская Федерация,Федеральный округ,Субъект РФ,Год,Месяц,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Российская Федерация,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,,Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,,,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
4,NaN,2022,2022,2022,2022,2022,2022,2022,2022,2022,...,2022,2022,2022,2022,2022,2022,2022,2022,2022,2022
5,"Товар, услуга",Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,...,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь,Июнь
6,Все товары и услуги,111.41,112.01,112.27,111.61,112.76,111.92,113.97,115.03,112.6,...,109.44,111.59,109.94,109.69,109.96,109.28,107.72,110.43,110.86,101.97
7,Все товары,112.71,113.03,114.23,113.05,114.97,113.88,115.66,116.48,113.48,...,110.27,113.35,112.41,111.36,111.88,111.96,109.27,112.55,112.83,103.63
8,Базовый индекс потребительских цен,113.97,114.55,115.17,114.56,116.17,115.37,117.54,118.12,115.58,...,110.78,113.38,112.19,112.48,112.29,113.76,109.26,113.04,113.24,104.84
9,Продовольственные товары,112.54,112.23,113.07,112.1,114.35,113.36,114.09,114.53,111.58,...,112.22,114.75,113.5,112.16,113.39,114.68,109.98,111.76,113.98,104.08


In [50]:
new_dataset_columns = [
	"Дата",
	"Страна",
	"Наименование округа",
	"Наименование субъекта",
	"Тип агрегации",
	"Категория товара или услуги",
	"Наименование товара",
	"Средняя цена",
	"Позиция в рейтинге",

	# *product_types_set
]

flat_dataset = {name: [] for name in new_dataset_columns}
area_names_set = set()
dates_set = set()
product_name_set = set()

def my_argsort(my_list, add=1):
	sorted_list = sorted(my_list)
	return [sorted_list.index(item)+add for item in my_list]

area_name = ""
for i in range(len(area_coords)):
	start_coords = area_coords[i]
	start_y, start_x = start_coords[1]
	end_y, end_x = start_coords[2]

	subject_name = dataset.iloc[start_y, start_x].strip()
	for col in range(start_x, end_x):
		# area_prices = {prod_type: [] for prod_type in product_types_set} 

		for j in range(6, len(dataset)):
			product_name = dataset.iloc[j, 0].strip()
			# print(product_name)
			if len(str(dataset.iloc[3, col])) > 0 and not dataset.iloc[3, col] is np.nan:
				area_name = dataset.iloc[3, col].strip()
			
			avg_price = dataset.iloc[j, col]
			date = str(dataset.iloc[5, col]).strip()
			# products_type = product_types_dict[product_name]
			flat_dataset["Дата"].append(date)
			flat_dataset["Страна"].append("Российская Федерация")
			flat_dataset["Наименование округа"].append(subject_name)
			flat_dataset["Наименование субъекта"].append(area_name)
			flat_dataset["Категория товара или услуги"].append("products_type")
			flat_dataset["Наименование товара"].append(product_name)
			flat_dataset["Средняя цена"].append(avg_price)
			flat_dataset["Позиция в рейтинге"].append("")

			area_names_set.add(subject_name)
			dates_set.add(date)
			product_name_set.add(product_name)


			if subject_name.strip() == "":
				flat_dataset["Тип агрегации"].append("Общее значение по РФ")
			
			if subject_name.strip() != "" and area_name.strip() == "":
				flat_dataset["Тип агрегации"].append("Общее значение по округу")

			if subject_name.strip() != "" and area_name.strip() != "":
				flat_dataset["Тип агрегации"].append("Общее значение по региону")
		# break
			# print(p_type, products_type)
			# for p_type in product_types_set:
			# 	if p_type.strip() == products_type.strip():
			# 		flat_dataset[products_type].append(products_type)
			# 	else:
			# 		flat_dataset[p_type].append("")


area_names_set = list(area_names_set)
dates_set = list(dates_set)
product_name_set = list(product_name_set)

p_data = pd.DataFrame(data=flat_dataset)

# for date in dates_set:
# 	for product_name in product_name_set:
# 		prices = p_data[(p_data['Наименование товара'] == product_name) & (p_data['Дата'] == date) & (p_data['Наименование субъекта'] != '')]['Средняя цена']
# 		prices_values = prices.values 
# 		prices_index = list(prices.index)
# 		non_nan_prices = {
# 			"prices": [],
# 			"indexes": []
# 		}
# 		for i, price in enumerate(prices_values):
# 			if not pd.isna(price):
# 				non_nan_prices['prices'].append(price)
# 				non_nan_prices['indexes'].append(prices_index[i])
# 		rating = my_argsort(non_nan_prices['prices'])
# 		prices_index = pd.Index(non_nan_prices['indexes'])
# 		p_data['Позиция в рейтинге'][prices_index] = rating
# 		# break
# 	# break
# p_data['Дата'] = pd.to_datetime(p_data['Дата'], format='%d.%m.%Y', errors='coerce')
# p_data.head(10)

In [51]:
p_data.to_excel("./test.xlsx", index=False, )

In [26]:
pd.to_datetime("15.04.2001", format='%d.%m.%Y', errors='coerce')

Timestamp('2001-04-15 00:00:00')

In [1]:
from pathlib import Path

p = Path("./data/test2/Средние_потребительские_цены_на_непродовольственные_товарыСредние.xlsx")

p.name

'Средние_потребительские_цены_на_непродовольственные_товарыСредние.xlsx'

In [10]:
print(Path(p.parent.absolute(), "parsed"))

d:\programming\AI\volgograd\data_processing\data_parser_10\data\test2\parsed
